## Урок 5. Рекуррентные нейронные сети
## Практическое задание

<ol>
    <li>Попробуйте изменить параметры нейронной сети, работающей с датасетом imdb, либо нейронной сети, работающей с airline-passengers (она прилагается вместе с датасетом к уроку в виде отдельного скрипта), так, чтобы улучшить ее точность. Приложите анализ.</li>
    <li>Попробуйте изменить параметры нейронной сети, генерирующей текст, таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившийся у вас текст и опишите то, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения.</li>
    <li>* Попробуйте на numpy реализовать нейронную сеть архитектуры. LSTM</li>
    <li>* Предложите свои варианты решения проблемы исчезающего градиента в RNN.</li>
</ol>

### Модель 1

In [4]:
%%time
from __future__ import print_function
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.utils import pad_sequences

max_features = 10000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 100
batch_size = 128 # увеличьте значение для ускорения обучения
def train_nn():
  print('Построение модели...')
  model = Sequential()
  model.add(Embedding(max_features, 128))
  model.add(LSTM(128, dropout=0.4, recurrent_dropout=0.4, implementation=2))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  print('Загрузка данных...')
 #стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров
  model.compile(loss='binary_crossentropy',
                optimizer='Adam', # при использовании этого оптимайзера модель показывает наилучшие результаты.
                metrics=['accuracy'])
  (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
  print(len(x_train), 'тренировочные последовательности')
  print(len(x_test), 'тестовые последовательности')

  print('Pad последовательности (примеров в x единицу времени)')
  x_train = pad_sequences(x_train, maxlen=maxlen)
  x_test = pad_sequences(x_test, maxlen=maxlen)
  print('x_train shape:', x_train.shape)
  print('x_test shape:', x_test.shape)

  print('Процесс обучения...')
  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=10, # увеличьте при необходимости
            validation_data=(x_test, y_test))
  score, acc = model.evaluate(x_test, y_test,
                              batch_size=batch_size)
  print('Результат при тестировании:', score)
  print('Тестовая точность:', acc)



train_nn()
  #

Построение модели...
Загрузка данных...
17464789/17464789 [==============================] - 0s 0us/step
25000 тренировочные последовательности
25000 тестовые последовательности
Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 100)
x_test shape: (25000, 100)
Процесс обучения...
Epoch 1/10
196/196 [==============================] - 122s 569ms/step - loss: 0.4540 - accuracy: 0.7720 - val_loss: 0.4506 - val_accuracy: 0.8254
Epoch 2/10
196/196 [==============================] - 100s 509ms/step - loss: 0.2868 - accuracy: 0.8838 - val_loss: 0.3378 - val_accuracy: 0.8515
Epoch 3/10
196/196 [==============================] - 102s 522ms/step - loss: 0.2180 - accuracy: 0.9136 - val_loss: 0.3562 - val_accuracy: 0.8457
Epoch 4/10
196/196 [==============================] - 98s 500ms/step - loss: 0.1800 - accuracy: 0.9319 - val_loss: 0.4746 - val_accuracy: 0.8362
Epoch 5/10
196/196 [==============================] - 97s 496ms/step - loss: 0.1391 - accuracy: 0.9480 - val_l

In [5]:
batch_size = 128
maxlen = 128
from keras.layers import Dropout

### Модель 2

In [6]:
%%time
batch_size = 64
epochs = 30
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.utils import pad_sequences
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.4, recurrent_dropout=0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='LeakyReLU'))
model.add(Dropout(0.4))
model.add(Dense(16, activation='elu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Use a different optimizer
model.compile(loss='binary_crossentropy',
              optimizer='RMSprop',
              metrics=['accuracy'])
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
# Train the model
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test))


Epoch 1/30
391/391 [==============================] - 255s 638ms/step - loss: 0.5322 - accuracy: 0.7290 - val_loss: 0.3626 - val_accuracy: 0.8394
Epoch 2/30
391/391 [==============================] - 248s 636ms/step - loss: 0.3613 - accuracy: 0.8560 - val_loss: 0.3376 - val_accuracy: 0.8576
Epoch 3/30
391/391 [==============================] - 251s 642ms/step - loss: 0.3058 - accuracy: 0.8828 - val_loss: 0.3637 - val_accuracy: 0.8405
Epoch 4/30
391/391 [==============================] - 260s 666ms/step - loss: 0.2741 - accuracy: 0.8977 - val_loss: 0.3164 - val_accuracy: 0.8626
Epoch 5/30
391/391 [==============================] - 253s 645ms/step - loss: 0.2462 - accuracy: 0.9064 - val_loss: 0.3249 - val_accuracy: 0.8649
Epoch 6/30
391/391 [==============================] - 240s 614ms/step - loss: 0.2241 - accuracy: 0.9175 - val_loss: 0.3454 - val_accuracy: 0.8510
Epoch 7/30
391/391 [==============================] - 243s 621ms/step - loss: 0.1977 - accuracy: 0.9289 - val_loss: 0.3834 -

### Модель 3

In [ ]:
%%time
from keras.backend import dropout
def train_nn_1():
  print('Загрузка данных...')
  (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
  print(len(x_train), 'тренировочные последовательности')
  print(len(x_test), 'тестовые последовательности')

  print('Pad последовательности (примеров в x единицу времени)')
  x_train = pad_sequences(x_train, maxlen=maxlen)
  x_test = pad_sequences(x_test, maxlen=maxlen)
  print('x_train shape:', x_train.shape)
  print('x_test shape:', x_test.shape)

  print('Построение модели...')
  model = Sequential()
  model.add(Embedding(max_features, 128))
  model.add(LSTM(128, dropout=0.4, recurrent_dropout=0.4))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(32, activation='LeakyReLU'))            # New line
  model.add(Dropout(0.4))
  model.add(Dense(16, activation='elu'))
  model.add(Dropout(0.2))                                  # New line
  model.add(Dense(1, activation='sigmoid'))

  # стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров
  model.compile(loss='binary_crossentropy',
                optimizer='RMSprop', # при использовании этого оптимайзера модель показывает наилучшие результаты.
                metrics=['accuracy'])
  callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
  print('Процесс обучения...')
  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=10, # увеличьте при необходимости
            callbacks = callbacks,
            validation_data=(x_test, y_test))
  score, acc = model.evaluate(x_test, y_test,
                              batch_size=batch_size)
  print('Результат при тестировании:', score)
  print('Тестовая точность:', acc)
train_nn_1()

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности
Pad последовательности (примеров в x единицу времени)


x_train shape: (25000, 128)
x_test shape: (25000, 128)
Построение модели...
Процесс обучения...
Epoch 1/10
98/98 [==============================] - 71s 681ms/step - loss: 0.6563 - accuracy: 0.6050 - val_loss: 0.6003 - val_accuracy: 0.6570
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


98/98 [==============================] - 67s 689ms/step - loss: 0.4686 - accuracy: 0.7908 - val_loss: 0.4005 - val_accuracy: 0.8260
Epoch 3/10
98/98 [==============================] - 65s 666ms/step - loss: 0.3833 - accuracy: 0.8430 - val_loss: 0.3837 - val_accuracy: 0.8297
Epoch 4/10
98/98 [==============================] - 66s 677ms/step - loss: 0.3377 - accuracy: 0.8662 - val_loss: 0.3501 - val_accuracy: 0.8455
Epoch 5/10
98/98 [==============================] - 64s 652ms/step - loss: 0.3048 - accuracy: 0.8845 - val_loss: 0.3416 - val_accuracy: 0.8534
Epoch 6/10
98/98 [==============================] - 62s 633ms/step - loss: 0.2832 - accuracy: 0.8914 - val_loss: 0.3803 - val_accuracy: 0.8324
Epoch 7/10
98/98 [==============================] - 66s 670ms/step - loss: 0.2617 - accuracy: 0.9030 - val_loss: 0.3465 - val_accuracy: 0.8634
Epoch 8/10
98/98 [==============================] - 64s 650ms/step - loss: 0.2399 - accuracy: 0.9121 - val_loss: 0.3527 - val_accuracy: 0.8612
Epoch 9/10

Для решения данной задачи лучше себя показала модель № 3 , Существенное увеличение эпох и уменьшение скорости обучения не существенно улучшает модель